## Librerias

In [14]:
import time
from datetime import datetime
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from selenium import webdriver
import pprint
import pandas as pd
import names as nm
import numpy as np
import traceback
import datetime


# Funciones 

##  Selenium

In [13]:
def click(buscador, driver, atributo="xpath"):
    """
    atributo : xpath, name, css, class, id
    """
    if atributo=="xpath":
        "dado un full xpath hace click"
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.XPATH, f"{buscador}")))
    elif atributo=="name":
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.NAME, f"{buscador}")))   
    
    elif atributo=="css":
        elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.CSS_SELECTOR, f"{buscador}")))
    
    elif atributo=="class":
         elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.CLASS_NAME, f"{buscador}")))
    elif atributo=="id":
         elemento  = WebDriverWait(driver, 15).until( \
            EC.presence_of_element_located((By.ID, f"{buscador}")))
    
        
    time.sleep(0.8)
    elemento.click()
    return elemento

def select_option(xpath_click, xpath_write, name, driver):
    """Selecciona el DROPDOWN, luego ESCRIBE name, FInalemente
    selecciona la primera opción, el xpath_write"""
    
    elemento  = WebDriverWait(driver, 15).until( \
    EC.presence_of_element_located((By.XPATH, f"{xpath_click}")))
    time.sleep(1)
    elemento.click()
    time.sleep(2)
    elemento2  = WebDriverWait(driver, 15).until( \
        EC.presence_of_element_located((By.XPATH, f"{xpath_write}")))
    time.sleep(2)
    elemento2.click()
    elemento2.send_keys(name, Keys.ENTER)



def escribir_y_enter(xpath, driver, texto="",sleep_time=0.8, atributo="xpath"):
    """
    atributo : xpath, name, css, class, id
    """
    elemento = click(xpath,driver, atributo=atributo)
    time.sleep(sleep_time)
    elemento.send_keys(texto, Keys.ENTER)

### BS 

In [ ]:
def soup_of_page(driver):
    time.sleep(2.5)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    return soup



### Algoritmos 

In [ ]:
def chequera_indice_provincia(html_tags, key):
    '''busca la provincia key en el array de html tags
    y devuelve el indice'''
    prov = key.strip().capitalize()
    for idx, provi in  enumerate(html_tags):
        res  = provi.get_text()
        res = str(res)
        res = res.strip().capitalize()
        if res == prov:
            return idx+1
    return "no encontrado"

### Generales

In [16]:
def norm_texto(string):
    nombre = str(string)
    nombre = nombre.strip().lower()
    return nombre


def generar_persona():
    '''return 
    sexo, nombre, apellido, email, numero'''
    data = {}
    #nombre y apeelido
    sexo =np.random.binomial(n=1, p=0.5)
    sexo = np.where(sexo==0, "male", "female" )
    random_number= np.random.randint(0,9999, 1)[0]
    nombre = nm.get_first_name(gender=sexo)
    apellido =  nm.get_last_name()
    fin_mail= "@gmail.com"
    email = nombre+apellido+str(random_number)+fin_mail
    email = email.lower()
    numero = np.random.randint(0,9,size=8)
    numero_str = ''.join([str(elem) for elem in numero])
    data["sexo"] = sexo
    data["nombre"] = nombre
    data["apellido"] = apellido
    data["email"] = email
    data ["numero" ] = numero_str
    return data

def calcular_edad(nacimiento_str):
    nacimiento_str = str(nacimiento_str)
    fecha =  datetime.datetime.strptime(nacimiento_str, '%d%m%Y').date()
    diff = datetime.date.today() - fecha
    edad = round(diff.days/365)
    return edad

## Comparar en Casa 

In [18]:

def get_url_perfil(marca, ano_vehiculo, modelo, version, codigo, edad, sexo, driver):
    driver.get('https://www.comparaencasa.com/')
    time.sleep(2)
    soup = soup_of_page(driver)


    #Marca
    marcas_tags = soup.find("select", {"id":"id_car_brand"})
    marcas  =  [x.get_text().strip().lower()  for x in marcas_tags.find_all("option")]
    idx_marca =  [i+1 for i, s in enumerate(marcas) if norm_texto(marca) in s][0]
    click(f"/html/body/div[1]/div[2]/div[8]/form/div/div[1]/select/option[{idx_marca}]", driver)



    #Año_vehiculo
    select_vehiculo = Select(driver.find_element_by_name('year'))
    select_vehiculo.select_by_value(ano_vehiculo)

    #continuar
    click("/html/body/div[1]/div[2]/div[8]/form/div/div[3]/button",driver)



    #MODELO
    titulo = driver.find_element_by_class_name("main-title")
    titulo.click()

    path_1 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[1]/div/div[1]/a"
    path_2 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[1]/div/div[1]/div/div[1]/input"
    select_option(path_1, path_2, modelo,driver)


    ##########AGREGAR OTRO METODO DE SELECION POR NOMBRE########################## y un try catch
    #VERSION
    time.sleep(2)
    titulo = driver.find_element_by_class_name("main-title")
    titulo.click()
    path_1 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[2]/div/div[1]/a"
    path_2 = "/html/body/div[1]/div[3]/div/div/form/div[1]/div[2]/div[2]/div/div[1]/div/div[1]/input"
    select_option(path_1, path_2, version,driver)
    
    time.sleep(3)

    #0KM
    #poor default es nuevo
    try:
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[3]/div/div[1]/label/div", driver)
    except:
        print("")
    
    #GNC
    #no por default
    path_no= "/html/body/div[1]/div[3]/div/div/form/div[1]/div[4]/div/div/label[2]/div"
    click(path_no,driver)

    #Codigo postal
    escribir_y_enter("id_zip_code", driver,texto=codigo, atributo="id")

    time.sleep(3)
    #Edad
    escribir_y_enter("age", driver, texto=edad, atributo="name")


    #Genero
    if sexo=="hombre":
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[6]/div[2]/div/label[1]/div", driver)
    else:
        click("/html/body/div[1]/div[3]/div/div/form/div[1]/div[6]/div[2]/div/label[2]/div", driver)

    try:
        persona = generar_persona()
        escribir_y_enter("email", driver, texto=persona["email"], atributo="name")
    except:
        print("")
    time.sleep(4)
    
    
    try:
        persona = generar_persona()
        escribir_y_enter("email", driver, texto=persona["email"], atributo="name")
    except:
        print("")
    return driver.current_url


In [17]:
def get_all_perfiles():
    perfiles = pd.read_excel("./perfiles.xlsx")
    tiempo_por_perfil = 40
    perfiles = pd.read_excel("./perfiles.xlsx")
    cantidad_periles= perfiles.shape[0]
    tiempo_estimado = tiempo_por_perfil*cantidad_periles 

    urls = {}
    print(f"Tiempo estimado total es: {round(tiempo_estimado/60,2)}, minutos")
    for i in range(perfiles.shape[0]):
        perfil = dict(perfiles.iloc[i])
        perfil["edad"] = calcular_edad(perfil["fecha (edad)"])
        print(f"*{perfil['id']} - ID, Obteniendo el perfil")
        options = Options()
        options.add_argument("--window-size=1980,1080")
        options.add_argument("start-maximized")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
        options.add_experimental_option('useAutomationExtension', False)
        #options.add_argument("--headless")
        driver = webdriver.Chrome(options=options)
        url = get_url_perfil(marca=perfil["marca"], ano_vehiculo=str(perfil["año"]), modelo=perfil["modelo"], \
                   version=perfil["version"], codigo= str(perfil["cp"]), edad=str(perfil["edad"]), sexo=str(perfil["sexo"]))
        urls[f"{perfil['id']}"] = url
    
    return urls
